In [21]:
import os
import pandas as pd
import numpy as np
import re
import itertools

In [22]:
def get_all_combinations(pattern):
    char_combinations = re.findall(r'\[([^]]+)\]', pattern)  # Find all character combinations within square brackets
    combinations = list(itertools.product(*char_combinations))  # Generate all combinations
    combinations = [''.join(combo) for combo in combinations]  # Convert combinations to strings
    return combinations

In [23]:
run_path = './'
data_path = os.path.join(run_path, 'final_files_oldinf')
libs_info_path = os.path.join(run_path, 'libs_info')
sample_info = pd.read_csv(os.path.join(libs_info_path,'samp_info.csv'))
sample_info.columns = [col.replace(' ', '_') for col in sample_info.columns]
libs_info = pd.read_csv(os.path.join(libs_info_path,'all_libs_TS.csv'), index_col=[0])
mut_dict = {'R':'[GA]', 'M':'[CA]', 'K':'[GT]', 'Y':'[ST]'}
data_files = os.listdir(data_path)
csv_dir = 'csv_files'
os.mkdir(csv_dir)
out_path = os.path.join(run_path, csv_dir)

FileExistsError: [Errno 17] File exists: 'csv_files'

In [16]:
data_files = os.listdir(data_path)
relevant_files = [file for file in data_files if '.txt' in file]
for file in relevant_files:
    if file not in relevant_files:
        continue
    curr_prom_number = file.split('.')[-2].split('_')[-1]
    sample_name = file.split('.')[0]
    pool_name = sample_name.split('_S')[0]
    well_barcode = int(file.split('.')[1])
    curr_samp_info = sample_info.query("Pool_name == @pool_name & Well_barcode_number == @well_barcode")
    curr_exp = curr_samp_info['Exp'].values[0]
    curr_tf = curr_samp_info['Strain'].values[0]
    curr_plasmid_pool = curr_samp_info['Plasmid_pool_number'].values[0]
    res_subset = sample_info.query("Strain == @curr_tf & Plasmid_pool_number == @curr_plasmid_pool & Exp == @curr_exp")
    res_files = []
    for row_i in res_subset.index:
        curr_row = res_subset.loc[row_i]
        composed_name = re.compile(curr_row['Pool_name'] + '_S[0-9]{1,2}' + '.' + str(curr_row['Well_barcode_number']) +
                                   '.prom_' + curr_prom_number +'.txt')
        res_files.append(list(filter(composed_name.match, relevant_files))[0])
    # generate all possible library sequences:
    curr_lib_info = libs_info.loc[int(curr_prom_number)]
    curr_lib_seq = curr_lib_info['Library_sequence']
    pattern = []
    for char in curr_lib_seq:
        if char in mut_dict.keys():
            pattern.append(mut_dict[char])
    str_pattern = ''.join(pattern)
    curr_lib_seqs = get_all_combinations(str_pattern)
    curr_res_table = pd.DataFrame(index=curr_lib_seqs)
    for file_i, curr_file in enumerate(res_files):
        curr_info = res_subset.iloc[file_i]
        curr_col_name = '_'. join([str(curr_info['Trans_repeat']), str(curr_info['Time_point']), str(curr_info['Time_point_repeat'])])
        curr_file_data = pd.read_csv(os.path.join(data_path, curr_file), sep=r'\s+', header=None, names=['read_num', 'seq'])
        read_counts = []
        for seq in curr_lib_seqs:
            if seq in curr_file_data['seq'].values:
                read_counts.append(curr_file_data.loc[curr_file_data['seq']==seq]['read_num'].values[0])
            else:
                read_counts.append(0)
        curr_res_table[curr_col_name] = read_counts
    ord_ids = np.argsort([int(x.split('_')[1]) for x in list(curr_res_table.columns)])
    curr_res_table_ordered = curr_res_table.iloc[:, ord_ids]    
    csv_name = '_'.join([curr_tf, curr_prom_number, str(curr_exp)])
    curr_res_table_ordered.to_csv(os.path.join(out_path, csv_name+'.csv'))
    relevant_files = np.setdiff1d(relevant_files, res_files)

In [ ]:
data_files = os.listdir(data_path)
relevant_files = [file for file in data_files if '.txt' in file]
for file in relevant_files:
    if file not in relevant_files:
        continue
    curr_prom_number = file.split('.')[-2].split('_')[-1]
    sample_name = file.split('.')[0]
    pool_name = sample_name.split('_S')[0]
    well_barcode = int(file.split('.')[1])
    curr_samp_info = sample_info.query("Pool_name == @pool_name & Well_barcode_number == @well_barcode")
    curr_exp = curr_samp_info['Exp'].values[0]
    curr_tf = curr_samp_info['Strain'].values[0]
    curr_plasmid_pool = curr_samp_info['Plasmid_pool_number'].values[0]
    res_subset = sample_info.query("Strain == @curr_tf & Plasmid_pool_number == @curr_plasmid_pool & Exp == @curr_exp")
    res_files = []
    for row_i in res_subset.index:
        curr_row = res_subset.loc[row_i]
        composed_name = re.compile(curr_row['Pool_name'] + '_S[0-9]{1,2}' + '.' + str(curr_row['Well_barcode_number']) +
                                   '.prom_' + curr_prom_number +'.txt')
        res_files.append(list(filter(composed_name.match, relevant_files))[0])
    # generate all possible library sequences:
    curr_lib_info = libs_info.loc[int(curr_prom_number)]
    curr_lib_seq = curr_lib_info['Library_sequence']
    pattern = []
    for char in curr_lib_seq:
        if char in mut_dict.keys():
            pattern.append(mut_dict[char])
    str_pattern = ''.join(pattern)
    curr_lib_seqs = get_all_combinations(str_pattern)
    curr_res_table = pd.DataFrame(index=curr_lib_seqs)
    for file_i, curr_file in enumerate(res_files):
        curr_info = res_subset.iloc[file_i]
        curr_col_name = '_'. join([str(curr_info['Trans_repeat']), str(curr_info['Time_point']), str(curr_info['Time_point_repeat'])])
        curr_file_data = pd.read_csv(os.path.join(data_path, curr_file), sep=r'\s+', header=None, names=['read_num', 'seq'])
        read_counts = []
        for seq in curr_lib_seqs:
            if seq in curr_file_data['seq'].values:
                read_counts.append(curr_file_data.loc[curr_file_data['seq']==seq]['read_num'].values[0])
            else:
                read_counts.append(0)
        curr_res_table[curr_col_name] = read_counts
    ord_ids = np.argsort([int(x.split('_')[1]) for x in list(curr_res_table.columns)])
    curr_res_table_ordered = curr_res_table.iloc[:, ord_ids]    
    csv_name = '_'.join([curr_tf, curr_prom_number, str(curr_exp)])
    break

In [29]:
curr_res_table[curr_col_name] = read_counts
curr_res_table = curr_res_table.sort_index(axis=1)
ord_ids = np.argsort([int(x.split('_')[1]) for x in list(curr_res_table.columns)])
curr_res_table_ordered = curr_res_table.iloc[:, ord_ids] 
curr_res_table_ordered
curr_res_table

,1_0_1,1_0_2,1_180_1,1_180_2,1_45_1,1_45_2,1_90_1,1_90_2,2_0_1,2_0_2,2_180_1,2_180_2,2_45_1,2_45_2,2_90_1,2_90_2
GGGGG,1367,965,1354,1303,1268,1150,1284,1254,1195,939,1208,1109,936,771,1302,1165
GGGGA,1341,1071,1250,1162,1864,1409,1400,1288,1162,1019,988,904,1066,975,1278,1198
GGGAG,1575,1242,1438,1454,1646,1533,1421,1139,1471,1267,1296,1119,1240,1050,1228,1336
GGGAA,1880,1374,1569,1304,1493,1425,1421,1399,1620,1183,1276,1175,1403,1298,1573,1569
GGAGG,2845,1706,2193,1647,2650,1979,1998,1698,1937,1807,1791,1593,1692,1535,1744,1706
GGAGA,2507,1666,2361,1844,2614,1862,2040,2100,2007,1832,1728,1621,1989,1587,1782,1887
GGAAG,3216,2108,2214,2276,3176,2437,2330,2124,2496,1945,1946,2193,2506,2212,2202,2185
GGAAA,3251,2125,2704,2501,3142,2344,2398,2062,2737,2158,2210,2049,2231,2228,2190,2533
GAGGG,1536,1015,1265,1181,1623,1244,1446,1239,1280,1023,1165,1013,1306,1053,1218,1125
GAGGA,1845,1206,1366,1366,1664,1296,1460,1200,1510,1109,1233,1076,1193,1185,1408,1383
